In [1]:
from __future__ import unicode_literals
import re, glob, codecs, os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, lemmatize
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [3]:
articles_filenames = sorted(glob.glob("./Text/*.txt"))

In [4]:

d = list()
for article in articles_filenames:
    with open(article, encoding='utf-8') as f:
        book = os.path.basename(article.split('.')[0])
        d.append(pd.DataFrame({'text': book, 'lines': f.readlines()}))
doc = pd.concat(d)
doc.tail(1000)

,text,lines
365,,Ignore such things as pork and pies.\n
366,,And when the moon begins to wane there are all...
367,,"One, he told me. There's a creek you passed ov..."
368,,And some foolish people still think they are a...
369,,"We left our buggy and slipped out, in a skiff,..."
370,,And as night drooped her mantle over everythin...
371,,IX. A Fortnight In Foster.\n
372,,HOW good it is to leave the conventions and ar...
373,,I arrived at Foster.\n
374,,"On the station, I was introduced to a local st..."


In [5]:
# Convert to list
data = doc.lines.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:10])

['Australian Essays ',
 'Francis W. L. Adams ',
 'Melbourne ',
 'William Inglis &amp; Co. ',
 '1886 ',
 'To Matthew Arnold in England. ',
 'Master, with this I send you, as a boy ',
 'that watches from below some cross-bow bird ',
 'swoop on his quarry carried up aloft, ',
 'and cries a cry of victory to his flight ']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:10])

[['australian', 'essays'], ['francis', 'adams'], ['melbourne'], ['william', 'inglis', 'amp', 'co'], [], ['to', 'matthew', 'arnold', 'in', 'england'], ['master', 'with', 'this', 'send', 'you', 'as', 'boy'], ['that', 'watches', 'from', 'below', 'some', 'cross', 'bow', 'bird'], ['swoop', 'on', 'his', 'quarry', 'carried', 'up', 'aloft'], ['and', 'cries', 'cry', 'of', 'victory', 'to', 'his', 'flight']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[10]]])


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['with', 'sheer', 'joy', 'of', 'achievementso', 'to', 'you']


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.max_length = 11000000
# def gensimlemm(texts):
#     texts_out = []
#     for sent in texts:
#         doc = " ".join(sent)
#         print(doc)
#         if len(doc) > 0:
#             lemmatized_out = [wd.decode('utf-8').split('/')[0] for wd in lemmatize(doc) if wd.decode('utf-8').split('/')[1]=='NN']
#             texts_out.append(lemmatized_out)
#     return texts_out
# # Do lemmatization keeping only noun
# data_lemmatized = gensimlemm(data_words_bigrams)

# Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

print(data_lemmatized[:10])

[['essay'], ['francis'], [], ['ingli'], [], ['england'], ['master', 'boy'], [], ['swoop', 'quarry'], ['cry', 'victory', 'flight']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


[[(0, 1)]]


In [12]:
id2word[100]

'intention'

In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:100]]

[[('essay', 1)],
 [('francis', 1)],
 [],
 [('ingli', 1)],
 [],
 [('england', 1)],
 [('boy', 1), ('master', 1)],
 [],
 [('quarry', 1), ('swoop', 1)],
 [('cry', 1), ('flight', 1), ('victory', 1)],
 [('joy', 1)],
 [('sea', 1), ('voice', 1)],
 [('wave', 1), ('wind', 1)],
 [('acknowledgment', 1)],
 [('honour', 1), ('truth', 1)],
 [],
 [('content', 1)],
 [('page', 1)],
 [('preface', 1)],
 [],
 [('civilization', 1)],
 [],
 [('gordon', 1), ('poetry', 1)],
 [],
 [('salvation_army', 1)],
 [],
 [('civilization', 1)],
 [],
 [('culture', 1)],
 [],
 [('dialogue', 1)],
 [('introduction', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('preface', 1)],
 [('honour', 1),
  ('dialogue', 1),
  ('acquaintancenay', 1),
  ('admiration', 1),
  ('analyst', 1),
  ('anyone', 1),
  ('artist', 1),
  ('attempt', 1),
  ('author', 2),
  ('book', 2),
  ('character', 3),
  ('charge', 1),
  ('considering', 1),
  ('critic', 1),
  ('criticism', 1),
  ('declare', 1),
  ('disraeli', 2),
  ('endymion'

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [16]:
# View the topics in LDA model
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.036*"life" + 0.029*"fact" + 0.026*"year" + 0.023*"voice" + '
  '0.023*"thought" + 0.022*"lady" + 0.022*"foot" + 0.021*"science" + '
  '0.020*"idea" + 0.016*"point"'),
 (1,
  '0.137*"man" + 0.031*"woman" + 0.021*"fire" + 0.020*"hill" + '
  '0.018*"bushranger" + 0.018*"native" + 0.017*"bush" + 0.017*"jerry" + '
  '0.014*"friend" + 0.014*"wind"'),
 (2,
  '0.060*"nothing" + 0.047*"night" + 0.034*"poet" + 0.022*"person" + '
  '0.018*"sun" + 0.017*"love" + 0.017*"instance" + 0.017*"road" + 0.016*"gold" '
  '+ 0.016*"death"'),
 (3,
  '0.055*"thing" + 0.027*"something" + 0.027*"country" + 0.019*"reason" + '
  '0.017*"spirit" + 0.016*"mark" + 0.015*"truth" + 0.014*"art" + '
  '0.013*"culture" + 0.013*"sense"'),
 (4,
  '0.038*"people" + 0.030*"place" + 0.028*"head" + 0.024*"book" + '
  '0.022*"literature" + 0.022*"work" + 0.019*"house" + 0.018*"study" + '
  '0.017*"name" + 0.014*"manner"'),
 (5,
  '0.062*"hand" + 0.042*"eye" + 0.032*"mind" + 0.025*"part" + 0.024*"power" + '
  '0.018*"r

In [17]:
#Compute Model Perplexity and Coherence Score
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.623644026469496

Coherence Score:  0.5683633240990822


In [18]:
# Visualize the topics-keywords
# Visualize the topics
num_topics=10
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
html_name ='LDA_Vis_with_'+str(num_topics)+'_Topics.html'
pyLDAvis.save_html(vis, html_name )
vis

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.052505 -0.186466       1        1  12.444360
4     -0.087714 -0.017033       2        1  11.084433
5      0.333583  0.210082       3        1  11.067852
3     -0.229074  0.279446       4        1  10.771854
0     -0.039885 -0.020380       5        1  10.208815
6     -0.008321 -0.095927       6        1  10.201482
9      0.002790 -0.045793       7        1   9.933575
8     -0.019377 -0.027789       8        1   9.222596
7     -0.000187 -0.050012       9        1   7.912094
2     -0.004320 -0.046128      10        1   7.152943, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
55    Default  13263.000000        man  13263.000000  30.0000  30.0000
73    Default   7399.000000       time   7399.000000  29.0000  29.0000
253   Default   5314.000000        way   5314.000000  28.0000  28.0000
49    Default   5354.000000       hand   5354.000000  27.0000  27.0000
124   Default   4595.000000      thing   4595.000000  26.0000  26.0000
87    Default   4397.000000        day   4397.000000  25.0000  25.0000
512   Default   3897.000000       word   3897.000000  24.0000  24.0000
477   Default   3325.000000    nothing   3325.000000  23.0000  23.0000
95    Default   3642.000000        eye   3642.000000  22.0000  22.0000
62    Default   3294.000000     people   3294.000000  21.0000  21.0000
686   Default   2621.000000      night   2621.000000  20.0000  20.0000
53    Default   2858.000000       life   2858.000000  19.0000  19.0000
757   Default   3060.000000      woman   3060.000000  18.0000  18.0000
1338  Default   2778.000000       mind   2778.000000  17.0000  17.0000
132   Default   2494.000000      world   2494.000000  16.0000  16.0000
108   Default   2569.000000      place   2569.000000  15.0000  15.0000
399   Default   2192.000000       side   2192.000000  14.0000  14.0000
843   Default   2417.000000       girl   2417.000000  13.0000  13.0000
502   Default   2391.000000       head   2391.000000  12.0000  12.0000
96    Default   2284.000000       fact   2284.000000  11.0000  11.0000
194   Default   2276.000000  something   2276.000000  10.0000  10.0000
447   Default   2258.000000    country   2258.000000   9.0000   9.0000
242   Default   1880.000000       poet   1880.000000   8.0000   8.0000
679   Default   1992.000000     nature   1992.000000   7.0000   7.0000
61    Default   2153.000000       part   2153.000000   6.0000   6.0000
256   Default   2076.000000       year   2076.000000   5.0000   5.0000
36    Default   2115.000000       book   2115.000000   4.0000   4.0000
752   Default   1985.000000       tree   1985.000000   3.0000   3.0000
347   Default   2042.000000      power   2042.000000   2.0000   2.0000
725   Default   1868.000000    history   1868.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
477   Topic10   3324.590820    nothing   3325.117676   2.6375  -2.8211
686   Topic10   2621.325439      night   2621.852295   2.6374  -3.0588
242   Topic10   1880.385498       poet   1880.913086   2.6374  -3.3910
1330  Topic10   1235.194092     person   1235.720947   2.6372  -3.8112
611   Topic10   1000.194702        sun   1000.721619   2.6371  -4.0223
506   Topic10    951.352417       love    951.879211   2.6371  -4.0723
283   Topic10    932.317871   instance    932.845154   2.6371  -4.0926
544   Topic10    928.324158       road    928.851074   2.6371  -4.0968
3093  Topic10    880.335205       gold    880.862061   2.6370  -4.1499
661   Topic10    880.232300      death    880.759094   2.6370  -4.1500
2568  Topic10    879.012817      table    879.539673   2.6370  -4.1514
2214  Topic10    840.968262   distance    841.495117   2.6370  -4.1957
674   Topic10    821.377747     savage    821.904663   2.6370  -4.2192
985   Topic10    798.769287      light    7

In [20]:
# Building LDA Mallet Model
mallet_path = os.getenv('MALLET_HOME')+'/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
mallet_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
vis_mallet = pyLDAvis.gensim.prepare(mallet_model, corpus, id2word)
html_name ='LDA_Vis_Mallet_with_'+str(num_topics)+'_Topics.html'
pyLDAvis.save_html(vis_mallet, html_name )
vis_mallet

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.000522  0.000337       1        1  11.266226
5      0.000224 -0.000572       2        1  10.670749
0     -0.000367  0.000644       3        1  10.350788
6      0.000338 -0.000348       4        1  10.065322
7     -0.000072  0.000287       5        1  10.041524
1      0.000435 -0.000470       6        1  10.020830
8     -0.000427  0.000625       7        1   9.598456
9     -0.000146 -0.000076       8        1   9.528759
2     -0.000998 -0.000669       9        1   9.422967
4      0.000491  0.000243      10        1   9.034380, topic_info=      Category       Freq                Term      Total  loglift  logprob
term                                                                      
15830  Default  21.000000            elegance  21.000000  30.0000  30.0000
19101  Default  20.000000          tenantless  20.000000  29.0000  29.0000
9491   Default  20.000000              castor  20.000000  28.0000  28.0000
4606   Default  22.000000                 sob  22.000000  27.0000  27.0000
10474  Default  20.000000              bowing  20.000000  26.0000  26.0000
28431  Default  21.000000                 tal  21.000000  25.0000  25.0000
25445  Default  21.000000            overleap  21.000000  24.0000  24.0000
36134  Default  21.000000       disconcerting  21.000000  23.0000  23.0000
24360  Default  21.000000            exporter  21.000000  22.0000  22.0000
36536  Default  21.000000            mckenzie  21.000000  21.0000  21.0000
15262  Default  21.000000        eccentricity  21.000000  20.0000  20.0000
17488  Default  20.000000           undergone  20.000000  19.0000  19.0000
22436  Default  21.000000            wooerone  21.000000  18.0000  18.0000
24847  Default  22.000000          distorting  22.000000  17.0000  17.0000
25823  Default  21.000000            katelook  21.000000  16.0000  16.0000
16970  Default  20.000000              bidder  20.000000  15.0000  15.0000
2112   Default  21.000000       south_america  21.000000  14.0000  14.0000
32009  Default  20.000000         nettlewhich  20.000000  13.0000  13.0000
11694  Default  21.000000             deadand  21.000000  12.0000  12.0000
6635   Default  22.000000                 wag  22.000000  11.0000  11.0000
10502  Default  20.000000               thata  20.000000  10.0000  10.0000
14244  Default  21.000000        advisability  21.000000   9.0000   9.0000
32466  Default  22.000000              renter  22.000000   8.0000   8.0000
3734   Default  20.000000           desultory  20.000000   7.0000   7.0000
2266   Default  20.000000          recreation  20.000000   6.0000   6.0000
28566  Default  21.000000              revoir  21.000000   5.0000   5.0000
34792  Default  21.000000           sydneyone  21.000000   4.0000   4.0000
23207  Default  20.000000             lumbago  20.000000   3.0000   3.0000
6280   Default  20.000000               blare  20.000000   2.0000   2.0000
35525  Default  21.000000            sebright  21.000000   1.0000   1.0000
...        ...        ...                 ...        ...      ...      ...
26040  Topic10   2.395056              sadowa  20.333644   0.2653 -10.2903
4874   Topic10   2.462827               strif  20.917871   0.2648 -10.2624
31824  Topic10   2.439831               rauca  20.726744   0.2646 -10.2718
10526  Topic10   2.386598               broil  20.289709   0.2639 -10.2939
22037  Topic10   2.449935           sojourner  20.848397   0.2629 -10.2677
71     Topic10   2.557136               state  21.764581   0.2627 -10.2249
7453   Topic10   2.496631               relic  21.280562   0.2613 -10.2488
18324  Topic10   2.416185               coyly  20.606745   0.2607 -10.2816
31676  Topic10   2.383197             peccavi  20.381626   0.2579 -10.2953
15858  Topic10   2.443767            helpmate  20.911382   0.2574 -10.2702
15830  Topic10   2.525880            elegance  21.617988   0.2572 -10.2372
11006  To

In [21]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('horse', 0.032761670383757595),
   ('tree', 0.022699889769027662),
   ('mile', 0.018406008869748008),
   ('bush', 0.017944576892511985),
   ('fire', 0.0166628214001897),
   ('side', 0.01648337563126458),
   ('river', 0.015945038324489222),
   ('ground', 0.014240303519700582),
   ('road', 0.01400958753108257),
   ('hill', 0.013907047091696788)]),
 (1,
  [('day', 0.052838836281262756),
   ('time', 0.03920768085975056),
   ('house', 0.03370253789724611),
   ('money', 0.017371491788381252),
   ('lady', 0.017068577223458756),
   ('work', 0.015501323604946727),
   ('mr', 0.015172068643074452),
   ('week', 0.013407262047439054),
   ('gentleman', 0.012011221009100606),
   ('evening', 0.011181498505182474)]),
 (2,
  [('night', 0.04420023375808888),
   ('day', 0.036517574617292395),
   ('hour', 0.019555859631118333),
   ('sun', 0.01567889620570712),
   ('air', 0.015536360785655236),
   ('light', 0.014510105761281679),
   ('bed', 0.014282049089198666),
   ('street', 0.013455343652897744)

In [ ]:
#How to find the optimal number of topics for LDA?
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
#18. Finding the dominant topic in each sentence
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.tail(20)

In [ ]:
#Find the most representative document for each topic
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
#Topic distribution across documents
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics